In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import load_model

In [2]:
word_index=imdb.get_word_index()
reverse_word_index = {value: key for key,value in word_index.items()}

In [3]:
model=load_model('imdb_rnn_model.h5')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
model.get_weights()

[array([[-0.0267138 , -0.04339178,  0.8720795 , ..., -0.15468185,
         -0.46381238, -0.02297038],
        [-0.03365148,  0.07020622,  0.01980266, ...,  0.0973582 ,
          0.07074029, -0.02998178],
        [ 0.0051808 ,  0.00767144, -0.01892059, ...,  0.01644322,
          0.02266905, -0.03700889],
        ...,
        [ 0.08600142, -0.14041461, -0.21900618, ..., -0.09190758,
          0.09492603,  0.15381815],
        [-0.11543469,  0.07411827,  0.08418993, ...,  0.07470294,
         -0.06175529, -0.060401  ],
        [ 0.11762854, -0.21059245,  0.15029375, ..., -0.0926768 ,
          0.06651253,  0.17038205]], shape=(10000, 128), dtype=float32),
 array([[-0.11096711, -0.07147659, -0.12083485, ..., -0.00116519,
          0.12694995, -0.11013351],
        [ 0.06899762,  0.08673579, -0.03819584, ..., -0.03184108,
          0.14881518, -0.19046977],
        [-0.06167074,  0.01010732, -0.11781693, ..., -0.03943426,
          0.00132097, -0.02182495],
        ...,
        [-0.0192836

In [5]:
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i-3, '?') for i in encoded_review])

def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word,2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

In [6]:
###Prediction function
def predict_review(review):
    preprocessed_review = preprocess_text(review)
    prediction = model.predict(preprocessed_review)

    sentiment = 'Positive' if prediction[0][0] >= 0.5 else 'Negative'

    return sentiment,prediction[0][0]

In [7]:
example_review = "The movie was fantastic! I really loved it and would watch it again."
sentiment,score = predict_review(example_review)

print(f"Review: {example_review}")
print(f"Predicted Sentiment: {sentiment} (Score: {score:.4f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
Review: The movie was fantastic! I really loved it and would watch it again.
Predicted Sentiment: Positive (Score: 0.8992)
